In [ ]:
import cv2
import dlib
from tensorflow.keras.models import load_model

# Load the CNN model from HDF5 file
CNNmodel = load_model('/Users/chittireddyruthwik/CNNmodel.h5')

# Load the face landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/chittireddyruthwik/Downloads/shape_predictor_68_face_landmarks.dat")

# Define class labels
class_labels = ['Drowsy', 'Non Drowsy']

# Capture and process video frames
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:  # Check if frame is captured successfully
        print("Failed to capture video frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        landmarks = predictor(gray, face)

        # Extract face region
        face_roi = gray[y1:y2, x1:x2]
        face_roi = cv2.resize(face_roi, (227, 227))  # Resize to match input shape of CNN model
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_GRAY2RGB)  # Convert to RGB
        face_roi = face_roi / 255.0  # Normalize pixel values to [0, 1]
        face_roi = face_roi.reshape((1, 227, 227, 3))  # Reshape to match input shape of CNN model

        # Predict using the loaded CNN model
        prediction = CNNmodel.predict(face_roi)
        prediction_label = class_labels[int(prediction[0][0] > 0.5)]  # Thresholding for binary classification

        # Draw prediction label on frame
        cv2.putText(frame, prediction_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow('Facial Landmarks', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2023-04-18 12:41:15.330168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 12:41:20.466778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 41ms/step
